In [14]:
import mir_eval
import librosa
import numpy as np
import argparse
import os
import glob
import demucs.separate
import csv


In [17]:
# get the source and separated audio files
def get_audio_files(current_dir):
    current_dir = current_dir

    filenames = ['other.wav', 'vocals.wav', 'bass.wav', 'drums.wav']
    file_paths = []
    for filename in filenames:
        files_found = glob.glob(os.path.join(current_dir, filename))
        file_paths.extend(files_found)

    # print("File paths found in the current directory:")
    # for path in file_paths:
    #     print(path)

    mixture_dir = os.path.join(current_dir, 'separated', 'mdx', 'mixture')
    mixture_file_paths = []

    for filename in filenames:
        files_found = glob.glob(os.path.join(mixture_dir, filename))
        mixture_file_paths.extend(files_found)

    # print("\nFile paths found in the mixture directory:")
    # for path in mixture_file_paths:
    #     print(path)

    return file_paths, mixture_file_paths
#get_audio_files("/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination")

In [18]:
# iterate through each directory, find the mixture file and create separated files
def find_mixture_wav_directories(directory):
    mixture_wav_directories = set()
    for root, dirs, files in os.walk(directory):
        if "separated" in dirs:
            pass
        else:
            for file in files:
                if file == "mixture.wav":
                    mixture_wav_directories.add(root)
                    break
    return mixture_wav_directories

directory_path = "/Users/kaimikkelsen/Downloads"
mixture_wav_directories = find_mixture_wav_directories(directory_path)

if mixture_wav_directories:
    counter = 0
    for directory in mixture_wav_directories:
        print(directory)
        os.chdir(directory)
        print(f"{counter} of {len(mixture_wav_directories)}")
        demucs.separate.main(["-n", "mdx", directory+"/mixture.wav"]) #call demucs separation command
        counter = counter + 1
else:
    print("No mixture.wav files found in the directory or its subdirectories.")


No mixture.wav files found in the directory or its subdirectories.


In [ ]:
# calculate the separation SDR values and write to a csv file

def getSeparationMetrics(audio_ref_list, audio_list):
    reference_sources = np.array(audio_ref_list)
    estimated_sources = np.array(audio_list)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, False)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def getSeparationMetricsFramewise(audio_ref_list, audio_list):
    reference_sources = np.array(audio_ref_list)
    estimated_sources = np.array(audio_list)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources_framewise(reference_sources, estimated_sources, window=22050, hop=11025)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def write_to_csv(data, file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Directory", "Other", "Vocals", "Bass", "Drums"])
        for row in data:
            writer.writerow(row)



directory_path = "/Users/kaimikkelsen/Downloads/test"
directories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

has_run = False
data_to_write = []
counter = 0
for directory in directories:
    reference_files, separated_files = get_audio_files(directory_path+"/"+directory)
    print(directory)
    print(f"{counter} of {len(directories)}")
    counter = counter + 1
    
    if(reference_files and separated_files and not has_run):

        row_data = [directory]
        for i in range(4):

            print(reference_files[i])
            print(separated_files[i])

            audio_ref, _ = librosa.load(reference_files[i])
            audio_sep, _ = librosa.load(separated_files[i])

            ref_list = [audio_ref]
            sep_list = [audio_sep]

            print(ref_list)

            sdr, sir, sar = getSeparationMetricsFramewise(ref_list, sep_list)
            print("SDR:", sdr)
            row_data.append(sdr)
        data_to_write.append(row_data)
        #has_run = True

csv_file_path = "/Users/kaimikkelsen/sdr_eval/demucs_output_framewise.csv"
write_to_csv(data_to_write, csv_file_path)
print("CSV file written successfully.")
    




In [ ]:

import os
import csv
import librosa

def getSeparationMetrics(audio_ref_list, audio_list):
    reference_sources = np.array(audio_ref_list)
    estimated_sources = np.array(audio_list)
    #print(reference_sources)
    #print(estimated_sources)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, False)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def write_to_csv(data, file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Directory", "Other", "Vocals", "Bass", "Drums"])
        for row in data:
            writer.writerow(row)


# Specify the directory path
#directory_path = '/path/to/your/directory'
directory_path = "/Users/kaimikkelsen/Downloads/test"
# Get a list of all directories in the specified directory
directories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

has_run = False
data_to_write = []
# Loop through each directory
counter = 0

other_ref = []
other_sep = []

vocals_ref = []
vocals_sep = []

bass_ref = []
bass_sep = []

drums_ref = []
drums_sep = []


for directory in directories:
    #print(directory)
    reference_files, separated_files = get_audio_files(directory_path+"/"+directory)
    #print(directory)
    print(f"{counter} of {len(directories)}")
    counter = counter + 1
    
    if(reference_files and separated_files and not has_run):

        row_data = [directory]
        for i in range(4):

           # print(reference_files[i])
           # print(separated_files[i])

            #audio_ref, sr1 = librosa.load(reference_files[i])
            #audio_sep, sr2 = librosa.load(separated_files[i])

            print(sr1)

            if i == 0: #other
                audio_ref, sr1 = librosa.load(reference_files[i])
                audio_sep, sr2 = librosa.load(separated_files[i])
                for j in range(0, len(audio_ref), sr1):
                    other_ref.append(audio_ref[j:j+sr1])
                    other_sep.append(audio_sep[j:j+sr1])
                #other_ref.append(audio_ref)
                #other_sep.append(audio_sep)

            # elif i == 1:#vocals
            #     vocals_ref.append(audio_ref)
            #     vocals_sep.append(audio_sep)
            # elif i == 2:#bass
            #     bass_ref.append(audio_ref)
            #     bass_sep.append(audio_sep)
            # elif i == 3:#drums
            #     drums_ref.append(audio_ref)
            #     drums_sep.append(audio_sep)
            # if counter == 3:
            #     has_run = True

    
    
print("sdr for other")
print(other_ref)
print(other_sep)
sdr, sir, sar = getSeparationMetrics(other_ref, other_sep)
print("SDR:", sdr)

# print("sdr for vocals")
# print(len(vocals_ref))
# sdr, sir, sar = getSeparationMetrics(vocals_ref, vocals_sep)
# print("SDR:", sdr)

# print("sdr for bass")
# print(len(bass_ref))
# sdr, sir, sar = getSeparationMetrics(bass_ref, bass_sep)
# print("SDR:", sdr)

# print("sdr for drums")
# print(len(drums_ref))
# sdr, sir, sar = getSeparationMetrics(drums_ref, drums_sep)
# print("SDR:", sdr)




In [ ]:

import os
import csv
import librosa
import numpy as np

def getSeparationMetrics(audio_ref_list, audio_list):
    reference_sources = np.array(audio_ref_list)
    estimated_sources = np.array(audio_list)
    #print(reference_sources)
    #print(estimated_sources)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, False)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def write_to_csv(data, file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Directory", "Other", "Vocals", "Bass", "Drums"])
        for row in data:
            writer.writerow(row)


# Specify the directory path
#directory_path = '/path/to/your/directory'
directory_path = "/Users/kaimikkelsen/Downloads/test"
# Get a list of all directories in the specified directory
directories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

has_run = False
data_to_write = []
# Loop through each directory
counter = 0

other_ref = []
other_sep = []

vocals_ref = []
vocals_sep = []

bass_ref = []
bass_sep = []

drums_ref = []
drums_sep = []


for directory in directories:
    print(directory)
    print(directory_path+"/"+directory)

    reference_files, separated_files = get_audio_files(directory_path+"/"+directory)
    #print(directory)
    print(f"{counter} of {len(directories)}")
    counter = counter + 1
    if(has_run):
        break
    
    if(reference_files and separated_files):

        row_data = [directory]
        for i in range(4):

           # print(reference_files[i])
           # print(separated_files[i])

            audio_ref, sr1 = librosa.load(reference_files[i])
            audio_sep, sr2 = librosa.load(separated_files[i])

            print(sr1)

            # Calculate the number of samples to iterate through
            num_samples = len(audio_ref)  # assuming both audios have the same length

            if i == 0: #other
                # Iterate through audio data by sr samples
                for j in range(0, num_samples, sr1):
                    start_sample = j
                    end_sample = min(j + sr1, num_samples) 
                    audio_ref_chunk = audio_ref[start_sample:end_sample]
                    audio_sep_chunk = audio_sep[start_sample:end_sample]

                    # Check if the chunk contains less than a second's worth of samples
                    if len(audio_ref_chunk) < sr1:
                        continue  # Skip this chunk and move to the next one
                    
                    other_ref.append([audio_ref_chunk])
                    other_sep.append([audio_sep_chunk])


                    #other_ref.append(audio_ref)
                    #other_sep.append(audio_sep)
            elif i == 1:#vocals
                vocals_ref.append(audio_ref)
                vocals_sep.append(audio_sep)
            elif i == 2:#bass
                bass_ref.append(audio_ref)
                bass_sep.append(audio_sep)
            elif i == 3:#drums
                drums_ref.append(audio_ref)
                drums_sep.append(audio_sep)

            has_run = True

    
    
print("sdr for other")
print(other_ref)
print(other_sep)
sdr, sir, sar = getSeparationMetrics(other_ref, other_sep)
print("SDR:", sdr)

# print("sdr for vocals")
# print(len(vocals_ref))
# sdr, sir, sar = getSeparationMetrics(vocals_ref, vocals_sep)
# print("SDR:", sdr)

# print("sdr for bass")
# print(len(bass_ref))
# sdr, sir, sar = getSeparationMetrics(bass_ref, bass_sep)
# print("SDR:", sdr)

# print("sdr for drums")
# print(len(drums_ref))
# sdr, sir, sar = getSeparationMetrics(drums_ref, drums_sep)
# print("SDR:", sdr)


